In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [2]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
funds = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

In [3]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [4]:
fundsSI =  funds[funds[' LIQUIDEZ MEDIA DIARIA'] > 250000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) ]
# fundsSI

In [5]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA', 'P/L' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA,P/L
1,VVEO3,2.03,15.209882,204.0,6.492553,1.01,10.18,2.01
2,LUPA3,1.55,10.550877,203.0,5.807017,1.33,3.72,1.16
3,HBOR3,2.06,9.502000,202.0,3.612621,0.38,10.56,5.38
4,HBRE3,5.16,22.461868,201.0,3.353075,1.14,19.67,4.54
5,SHOW3,1.52,6.105755,200.0,3.016944,0.63,2.63,2.43
6,ETER3,5.89,21.034793,199.0,2.571272,1.71,11.50,3.45
7,MTRE3,3.36,11.970620,198.0,2.562685,0.69,9.23,4.90
8,ALLD3,7.18,25.199554,197.0,2.509687,1.67,16.90,4.30
9,EUCA4,14.73,50.730169,196.0,2.444003,4.30,26.60,3.42
10,BRSR6,10.99,36.772279,195.0,2.345976,2.46,24.43,4.47


In [6]:
fundsSI =  fundsSI[(fundsSI['P/L'] > 0)]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,VVEO3,2.03,15.209882,204.0,6.492553,1.01,10.18
2,LUPA3,1.55,10.550877,203.0,5.807017,1.33,3.72
3,HBOR3,2.06,9.502000,202.0,3.612621,0.38,10.56
4,HBRE3,5.16,22.461868,201.0,3.353075,1.14,19.67
5,SHOW3,1.52,6.105755,200.0,3.016944,0.63,2.63
6,ETER3,5.89,21.034793,199.0,2.571272,1.71,11.50
7,MTRE3,3.36,11.970620,198.0,2.562685,0.69,9.23
8,ALLD3,7.18,25.199554,197.0,2.509687,1.67,16.90
9,EUCA4,14.73,50.730169,196.0,2.444003,4.30,26.60
10,BRSR6,10.99,36.772279,195.0,2.345976,2.46,24.43


In [7]:
fundsSI =  fundsSI[(fundsSI['CAGR RECEITAS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
3,HBOR3,2.06,9.502000,202.0,3.612621,0.38,10.56
4,HBRE3,5.16,22.461868,201.0,3.353075,1.14,19.67
5,SHOW3,1.52,6.105755,200.0,3.016944,0.63,2.63
6,ETER3,5.89,21.034793,199.0,2.571272,1.71,11.50
7,MTRE3,3.36,11.970620,198.0,2.562685,0.69,9.23
8,ALLD3,7.18,25.199554,197.0,2.509687,1.67,16.90
9,EUCA4,14.73,50.730169,196.0,2.444003,4.30,26.60
10,BRSR6,10.99,36.772279,195.0,2.345976,2.46,24.43
11,COCE5,29.90,97.739130,194.0,2.268867,7.65,55.50
12,SOMA3,5.86,18.293851,193.0,2.121818,2.01,7.40


In [8]:
fundsSI =  fundsSI[(fundsSI['CAGR LUCROS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
5,SHOW3,1.52,6.105755,200.0,3.016944,0.63,2.63
7,MTRE3,3.36,11.970620,198.0,2.562685,0.69,9.23
8,ALLD3,7.18,25.199554,197.0,2.509687,1.67,16.90
9,EUCA4,14.73,50.730169,196.0,2.444003,4.30,26.60
10,BRSR6,10.99,36.772279,195.0,2.345976,2.46,24.43
11,COCE5,29.90,97.739130,194.0,2.268867,7.65,55.50
12,SOMA3,5.86,18.293851,193.0,2.121818,2.01,7.40
13,BMGB4,3.10,9.531317,192.0,2.074618,0.56,7.21
15,JHSF3,3.85,11.172735,190.0,1.902009,0.76,7.30
16,ALOS3,20.83,58.674950,189.0,1.816848,5.97,25.63


In [9]:
fundsSI =  fundsSI[(fundsSI['DIVIDA LIQUIDA / EBIT'] <4) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
5,SHOW3,1.52,6.105755,200.0,3.016944,0.63,2.63
8,ALLD3,7.18,25.199554,197.0,2.509687,1.67,16.90
9,EUCA4,14.73,50.730169,196.0,2.444003,4.30,26.60
11,COCE5,29.90,97.739130,194.0,2.268867,7.65,55.50
12,SOMA3,5.86,18.293851,193.0,2.121818,2.01,7.40
15,JHSF3,3.85,11.172735,190.0,1.902009,0.76,7.30
16,ALOS3,20.83,58.674950,189.0,1.816848,5.97,25.63
19,GOAU4,10.34,28.314025,186.0,1.738300,2.08,17.13
20,GOAU3,10.43,28.314025,185.0,1.714672,2.08,17.13
22,VLID3,15.18,39.507847,183.0,1.602625,3.69,18.80


In [10]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
5,SHOW3,1.52,6.105755,200.0,3.016944,0.63,2.63
8,ALLD3,7.18,25.199554,197.0,2.509687,1.67,16.90
9,EUCA4,14.73,50.730169,196.0,2.444003,4.30,26.60
11,COCE5,29.90,97.739130,194.0,2.268867,7.65,55.50
12,SOMA3,5.86,18.293851,193.0,2.121818,2.01,7.40
15,JHSF3,3.85,11.172735,190.0,1.902009,0.76,7.30
16,ALOS3,20.83,58.674950,189.0,1.816848,5.97,25.63
19,GOAU4,10.34,28.314025,186.0,1.738300,2.08,17.13
22,VLID3,15.18,39.507847,183.0,1.602625,3.69,18.80
23,NEOE3,18.16,46.570495,182.0,1.564455,3.97,24.28


In [11]:
# linhas_com_nan = top20[top20.isna().any(axis=1)]
# linhas_com_nan

### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.